<a href="https://colab.research.google.com/github/hyunholee26/Short-term-water-level-prediction-in-the-river-using-LSTM-with-MC-dropout/blob/main/collect_data/%5Bwlp%5D_1_get_wamis_openapi_water_level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

from urllib.request import Request, urlopen, build_opener, HTTPRedirectHandler, addinfourl
from urllib.parse import urlencode, quote_plus

from bs4 import BeautifulSoup

In [ ]:
path = '/content/drive/MyDrive/data/flood/'

In [ ]:
url = 'http://www.wamis.go.kr:8080/wamis/openapi/wkw/wl_dubwlobs'
queryParams = '?' + urlencode({ quote_plus('key') : '36c9f67744f056ef568d114e3f9187aadda8173c82', quote_plus('oper') : 'y' })
request = Request(url + queryParams)
request.get_method = lambda: 'GET'
response_body = urlopen(request).read().decode('utf-8')
response_body
#soup = BeautifulSoup(response_body, "html.parser")

'{"result":{"code":"success","msg":"완료되었습니다!"},"count":799,"list":[{"bbsnnm":"한강","obscd":"1001602","obsnm":"평창군(송정교)","clsyn":"운영","obsknd":"T/M","sbsncd":"100108","mngorg":"환경부"},{"bbsnnm":"한강","obscd":"1001603","obsnm":"태백시(무사교)","clsyn":"운영","obsknd":"T/M","sbsncd":"100101","mngorg":"한국수자원공사"},{"bbsnnm":"한강","obscd":"1001605","obsnm":"송천1교","clsyn":"운영","obsknd":"T/M","sbsncd":"100105","mngorg":"한국수력원자력"},{"bbsnnm":"한강","obscd":"1001607","obsnm":"삼척시(번천교)","clsyn":"운영","obsknd":"T/M","sbsncd":"100101","mngorg":"한국수자원공사"},{"bbsnnm":"한강","obscd":"1001613","obsnm":"삼척시(광동교)","clsyn":"운영","obsknd":"T/M","sbsncd":"100102","mngorg":"한국수자원공사"},{"bbsnnm":"한강","obscd":"1001615","obsnm":"삼척시(갈밭교)","clsyn":"운영","obsknd":"T/M","sbsncd":"100102","mngorg":"한국수자원공사"},{"bbsnnm":"한강","obscd":"1001620","obsnm":"정선군(송계교)","clsyn":"운영","obsknd":"T/M","sbsncd":"100103","mngorg":"환경부"},{"bbsnnm":"한강","obscd":"1001622","obsnm":"정선군(혈천교)","clsyn":"운영","obsknd":"T/M","sbsncd":"100102","mngorg":"환경부"},{"bbs

In [ ]:
import json

jj = json.loads(response_body)
jj['list']

[{'bbsnnm': '한강',
  'clsyn': '운영',
  'mngorg': '환경부',
  'obscd': '1001602',
  'obsknd': 'T/M',
  'obsnm': '평창군(송정교)',
  'sbsncd': '100108'},
 {'bbsnnm': '한강',
  'clsyn': '운영',
  'mngorg': '한국수자원공사',
  'obscd': '1001603',
  'obsknd': 'T/M',
  'obsnm': '태백시(무사교)',
  'sbsncd': '100101'},
 {'bbsnnm': '한강',
  'clsyn': '운영',
  'mngorg': '한국수력원자력',
  'obscd': '1001605',
  'obsknd': 'T/M',
  'obsnm': '송천1교',
  'sbsncd': '100105'},
 {'bbsnnm': '한강',
  'clsyn': '운영',
  'mngorg': '한국수자원공사',
  'obscd': '1001607',
  'obsknd': 'T/M',
  'obsnm': '삼척시(번천교)',
  'sbsncd': '100101'},
 {'bbsnnm': '한강',
  'clsyn': '운영',
  'mngorg': '한국수자원공사',
  'obscd': '1001613',
  'obsknd': 'T/M',
  'obsnm': '삼척시(광동교)',
  'sbsncd': '100102'},
 {'bbsnnm': '한강',
  'clsyn': '운영',
  'mngorg': '한국수자원공사',
  'obscd': '1001615',
  'obsknd': 'T/M',
  'obsnm': '삼척시(갈밭교)',
  'sbsncd': '100102'},
 {'bbsnnm': '한강',
  'clsyn': '운영',
  'mngorg': '환경부',
  'obscd': '1001620',
  'obsknd': 'T/M',
  'obsnm': '정선군(송계교)',
  'sbsncd': '100103'

In [ ]:
df = pd.json_normalize(jj['list'])
df

,bbsnnm,obscd,obsnm,clsyn,obsknd,sbsncd,mngorg
0,한강,1001602,평창군(송정교),운영,T/M,100108,환경부
1,한강,1001603,태백시(무사교),운영,T/M,100101,한국수자원공사
2,한강,1001605,송천1교,운영,T/M,100105,한국수력원자력
3,한강,1001607,삼척시(번천교),운영,T/M,100101,한국수자원공사
4,한강,1001613,삼척시(광동교),운영,T/M,100102,한국수자원공사
...,...,...,...,...,...,...,...
794,만경ㆍ동진,8000001,대아저수지,운영,T/M,330101,환경부
795,만경ㆍ동진,8000002,경천저수지,운영,T/M,330102,환경부
796,영산강,8000005,수양저수지,운영,T/M,500203,환경부
797,섬진강,8888888,화순군(동복댐하),운영,T/M,400704,환경부


In [ ]:
class NoRedirectHandler(HTTPRedirectHandler):
    def http_error_302(self, req, fp, code, msg, headers):
        infourl = addinfourl(fp, headers, req.get_full_url())
        infourl.status = code
        infourl.code = code
        print('exception')
        raise Exception('redirection')
        return infourl
    http_error_300 = http_error_302
    http_error_301 = http_error_302
    http_error_303 = http_error_302
    http_error_307 = http_error_302

url = 'http://www.wamis.go.kr:8080/wamis/openapi/wkw/wl_obsinfo'
df_detail = pd.DataFrame()
for i, row in df.iterrows():
  obscd = row['obscd'] 
  queryParams = '?' + urlencode({ quote_plus('key') : '36c9f67744f056ef568d114e3f9187aadda8173c82', quote_plus('obscd') : str(obscd) })
  request = Request(url + queryParams)
  request.get_method = lambda: 'GET'
  opener = build_opener(NoRedirectHandler())

  trycount = 10 # retry counter 
  while trycount > 0 :
    try:
      response_body = opener.open(request).read().decode('utf-8')
      break;
    except:
      trycount = trycount - 1
      continue;
  #response_body = urlopen(request).read().decode('utf-8')

  if trycount > 0:
    print(response_body)
    jj = json.loads(response_body)
    if i == 0:
      df_detail = pd.json_normalize(jj['list'])
    else:
      df_detail = df_detail.append(pd.json_normalize(jj['list']))
    #display(df_detail)

exception
{"result":{"code":"success","msg":"완료되었습니다!"},"count":1,"list":[{"obsnm":"평창군(송정교)","obsnmeng":"Pyeongchanggun(Songjeonggyo)","wlobscd":"1001602","mggvcd":"환경부","bbsncd":"한강","sbsncd":"100108","obsopndt":"2015-12-30","obskdcd":"T/M","rivnm":"오대천","bsnara":null,"rvwdt":null,"bedslp":null,"rvmjctdis":null,"wsrdis":null,"addr":"강원도 평창군 진부면 하진부리 송정교","lon":"128-33-04","lat":"37-37-27 ","tmx":"336935","tmy":"459417","gdt":"511.589","wltel":"520.43","tdeyn":"N","mxgrd":"5.9","sistartobsdh":"201512","siendobsdh":"202107","olstartobsdh":"201512","olendobsdh":"202107"}]}
{"result":{"code":"success","msg":"완료되었습니다!"},"count":1,"list":[{"obsnm":"태백시(무사교)","obsnmeng":"Taebaeksi(Musagyo)","wlobscd":"1001603","mggvcd":"한국수자원공사","bbsncd":"한강","sbsncd":"100101","obsopndt":"2005-05-01","obskdcd":"T/M","rivnm":null,"bsnara":null,"rvwdt":null,"bedslp":null,"rvmjctdis":null,"wsrdis":null,"addr":"강원도 태백시 상사미동 ","lon":"128-59-40","lat":"37-18-54 ","tmx":"376803","tmy":"425838","gdt":null,"wltel":n

In [ ]:
df_detail

,obsnm,obsnmeng,wlobscd,mggvcd,bbsncd,sbsncd,obsopndt,obskdcd,rivnm,bsnara,rvwdt,bedslp,rvmjctdis,wsrdis,addr,lon,lat,tmx,tmy,gdt,wltel,tdeyn,mxgrd,sistartobsdh,siendobsdh,olstartobsdh,olendobsdh
0,평창군(송정교),Pyeongchanggun(Songjeonggyo),1001602,환경부,한강,100108,2015-12-30,T/M,오대천,None,None,None,None,None,강원도 평창군 진부면 하진부리 송정교,128-33-04,37-37-27,336935,459417,511.589,520.43,N,5.9,201512,202107,201512,202107
0,태백시(무사교),Taebaeksi(Musagyo),1001603,한국수자원공사,한강,100101,2005-05-01,T/M,None,None,None,None,None,None,강원도 태백시 상사미동,128-59-40,37-18-54,376803,425838,None,None,N,None,201508,202107,201508,202107
0,송천1교,Song Cheon 1 Kyo,1001605,한국수력원자력,한강,100105,2013-11-30,T/M,송천,123.45,None,None,3.7,None,강원도 평창군 대관령면 횡계리,128-42-35,37-40-05,350849,464532,724.782,731.705,N,None,201311,202107,201311,202107
0,삼척시(번천교),Samcheoksi(Bunchungyo),1001607,한국수자원공사,한강,100101,2006-11-01,T/M,None,None,None,None,None,None,강원도 삼척시 하장면 번천리 천375,128-59-47,37-21-52,376860,431330,None,None,N,None,201508,202107,201508,202107
0,삼척시(광동교),Samcheoksi(Gwangdonggyo),1001613,한국수자원공사,한강,100102,2013-04-01,T/M,None,None,None,None,None,None,강원도 삼척시 하장면 광동리 천261,128-56-56,37-20-91,372665,430594,None,None,N,None,201508,202107,201508,202107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,대아저수지,None,8000001,환경부,만경ㆍ동진,330101,2020-01-01,T/M,None,None,None,None,None,None,,None,--,None,None,None,None,N,None,202003,202107,202003,202107
0,경천저수지,None,8000002,환경부,만경ㆍ동진,330102,2020-01-01,T/M,None,None,None,None,None,None,,None,--,None,None,None,None,N,None,202003,202107,202003,202107
0,수양저수지,None,8000005,환경부,영산강,500203,2020-03-05,T/M,동진강,None,None,None,None,None,,None,--,None,None,None,None,N,None,202003,202107,202003,202107
0,화순군(동복댐하),None,8888888,환경부,섬진강,400704,2020-01-01,T/M,동복천,None,None,None,None,None,,None,--,None,None,None,None,N,None,202004,202107,202004,202107


In [ ]:
df_detail.shape

(799, 27)

In [ ]:
df_final = pd.merge(df, df_detail, left_on='obscd', right_on='wlobscd', how='left')
df_final

,bbsnnm,obscd,obsnm_x,clsyn,obsknd,sbsncd_x,mngorg,obsnm_y,obsnmeng,wlobscd,mggvcd,bbsncd,sbsncd_y,obsopndt,obskdcd,rivnm,bsnara,rvwdt,bedslp,rvmjctdis,wsrdis,addr,lon,lat,tmx,tmy,gdt,wltel,tdeyn,mxgrd,sistartobsdh,siendobsdh,olstartobsdh,olendobsdh
0,한강,1001602,평창군(송정교),운영,T/M,100108,환경부,평창군(송정교),Pyeongchanggun(Songjeonggyo),1001602,환경부,한강,100108,2015-12-30,T/M,오대천,None,None,None,None,None,강원도 평창군 진부면 하진부리 송정교,128-33-04,37-37-27,336935,459417,511.589,520.43,N,5.9,201512,202107,201512,202107
1,한강,1001603,태백시(무사교),운영,T/M,100101,한국수자원공사,태백시(무사교),Taebaeksi(Musagyo),1001603,한국수자원공사,한강,100101,2005-05-01,T/M,None,None,None,None,None,None,강원도 태백시 상사미동,128-59-40,37-18-54,376803,425838,None,None,N,None,201508,202107,201508,202107
2,한강,1001605,송천1교,운영,T/M,100105,한국수력원자력,송천1교,Song Cheon 1 Kyo,1001605,한국수력원자력,한강,100105,2013-11-30,T/M,송천,123.45,None,None,3.7,None,강원도 평창군 대관령면 횡계리,128-42-35,37-40-05,350849,464532,724.782,731.705,N,None,201311,202107,201311,202107
3,한강,1001607,삼척시(번천교),운영,T/M,100101,한국수자원공사,삼척시(번천교),Samcheoksi(Bunchungyo),1001607,한국수자원공사,한강,100101,2006-11-01,T/M,None,None,None,None,None,None,강원도 삼척시 하장면 번천리 천375,128-59-47,37-21-52,376860,431330,None,None,N,None,201508,202107,201508,202107
4,한강,1001613,삼척시(광동교),운영,T/M,100102,한국수자원공사,삼척시(광동교),Samcheoksi(Gwangdonggyo),1001613,한국수자원공사,한강,100102,2013-04-01,T/M,None,None,None,None,None,None,강원도 삼척시 하장면 광동리 천261,128-56-56,37-20-91,372665,430594,None,None,N,None,201508,202107,201508,202107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794,만경ㆍ동진,8000001,대아저수지,운영,T/M,330101,환경부,대아저수지,None,8000001,환경부,만경ㆍ동진,330101,2020-01-01,T/M,None,None,None,None,None,None,,None,--,None,None,None,None,N,None,202003,202107,202003,202107
795,만경ㆍ동진,8000002,경천저수지,운영,T/M,330102,환경부,경천저수지,None,8000002,환경부,만경ㆍ동진,330102,2020-01-01,T/M,None,None,None,None,None,None,,None,--,None,None,None,None,N,None,202003,202107,202003,202107
796,영산강,8000005,수양저수지,운영,T/M,500203,환경부,수양저수지,None,8000005,환경부,영산강,500203,2020-03-05,T/M,동진강,None,None,None,None,None,,None,--,None,None,None,None,N,None,202003,202107,202003,202107
797,섬진강,8888888,화순군(동복댐하),운영,T/M,400704,환경부,화순군(동복댐하),None,8888888,환경부,섬진강,400704,2020-01-01,T/M,동복천,None,None,None,None,None,,None,--,None,None,None,None,N,None,202004,202107,202004,202107


In [ ]:
df_final['lon_new'] = pd.to_numeric(df_final.lon.str.split('-').str[0], errors='coerce') + pd.to_numeric(df_final.lon.str.split('-').str[1], errors='coerce') / 60 + pd.to_numeric(df_final.lon.str.split('-').str[2], errors='coerce') / 3600
df_final['lat_new'] = pd.to_numeric(df_final.lat.str.split('-').str[0], errors='coerce') + pd.to_numeric(df_final.lat.str.split('-').str[1], errors='coerce') / 60 + pd.to_numeric(df_final.lat.str.split('-').str[2], errors='coerce') / 3600
df_final

,bbsnnm,obscd,obsnm_x,clsyn,obsknd,sbsncd_x,mngorg,obsnm_y,obsnmeng,wlobscd,mggvcd,bbsncd,sbsncd_y,obsopndt,obskdcd,rivnm,bsnara,rvwdt,bedslp,rvmjctdis,wsrdis,addr,lon,lat,tmx,tmy,gdt,wltel,tdeyn,mxgrd,sistartobsdh,siendobsdh,olstartobsdh,olendobsdh,lon_new,lat_new
0,한강,1001602,평창군(송정교),운영,T/M,100108,환경부,평창군(송정교),Pyeongchanggun(Songjeonggyo),1001602,환경부,한강,100108,2015-12-30,T/M,오대천,None,None,None,None,None,강원도 평창군 진부면 하진부리 송정교,128-33-04,37-37-27,336935,459417,511.589,520.43,N,5.9,201512,202107,201512,202107,128.551111,37.624167
1,한강,1001603,태백시(무사교),운영,T/M,100101,한국수자원공사,태백시(무사교),Taebaeksi(Musagyo),1001603,한국수자원공사,한강,100101,2005-05-01,T/M,None,None,None,None,None,None,강원도 태백시 상사미동,128-59-40,37-18-54,376803,425838,None,None,N,None,201508,202107,201508,202107,128.994444,37.315000
2,한강,1001605,송천1교,운영,T/M,100105,한국수력원자력,송천1교,Song Cheon 1 Kyo,1001605,한국수력원자력,한강,100105,2013-11-30,T/M,송천,123.45,None,None,3.7,None,강원도 평창군 대관령면 횡계리,128-42-35,37-40-05,350849,464532,724.782,731.705,N,None,201311,202107,201311,202107,128.709722,37.668056
3,한강,1001607,삼척시(번천교),운영,T/M,100101,한국수자원공사,삼척시(번천교),Samcheoksi(Bunchungyo),1001607,한국수자원공사,한강,100101,2006-11-01,T/M,None,None,None,None,None,None,강원도 삼척시 하장면 번천리 천375,128-59-47,37-21-52,376860,431330,None,None,N,None,201508,202107,201508,202107,128.996389,37.364444
4,한강,1001613,삼척시(광동교),운영,T/M,100102,한국수자원공사,삼척시(광동교),Samcheoksi(Gwangdonggyo),1001613,한국수자원공사,한강,100102,2013-04-01,T/M,None,None,None,None,None,None,강원도 삼척시 하장면 광동리 천261,128-56-56,37-20-91,372665,430594,None,None,N,None,201508,202107,201508,202107,128.948889,37.358611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794,만경ㆍ동진,8000001,대아저수지,운영,T/M,330101,환경부,대아저수지,None,8000001,환경부,만경ㆍ동진,330101,2020-01-01,T/M,None,None,None,None,None,None,,None,--,None,None,None,None,N,None,202003,202107,202003,202107,NaN,NaN
795,만경ㆍ동진,8000002,경천저수지,운영,T/M,330102,환경부,경천저수지,None,8000002,환경부,만경ㆍ동진,330102,2020-01-01,T/M,None,None,None,None,None,None,,None,--,None,None,None,None,N,None,202003,202107,202003,202107,NaN,NaN
796,영산강,8000005,수양저수지,운영,T/M,500203,환경부,수양저수지,None,8000005,환경부,영산강,500203,2020-03-05,T/M,동진강,None,None,None,None,None,,None,--,None,None,None,None,N,None,202003,202107,202003,202107,NaN,NaN
797,섬진강,8888888,화순군(동복댐하),운영,T/M,400704,환경부,화순군(동복댐하),None,8888888,환경부,섬진강,400704,2020-01-01,T/M,동복천,None,None,None,None,None,,None,--,None,None,None,None,N,None,202004,202107,202004,202107,NaN,NaN


In [ ]:
df_final.to_csv(path + 'stage.csv', index=False)

In [ ]:
import folium

#지도의 중심을 지정하기 위해 위도와 경도의 평균 구하기
lat = df_final['lat_new'].mean()
lon = df_final['lon_new'].mean()

m = folium.Map([lat,lon],zoom_start=9)

for i in df_final.index:
    sub_lat = df_final.loc[i,'lat_new']
    sub_lon = df_final.loc[i,'lon_new']

    if (np.isnan(sub_lat) == False) & (np.isnan(sub_lat) == False) :
      title = df_final.loc[i,'obsnm_x'] + '(' + df_final.loc[i,'obscd'] + ')'
      folium.Marker([sub_lat,sub_lon],tooltip = title).add_to(m)

#한글이 안나오는 오류로 html로 trouble shooting 
m.save(path + 'example.html')
m

